In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!unzip "/content/drive/MyDrive/archive (1).zip"

Streaming output truncated to the last 5000 lines.
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keloids-8.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keloids-80.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keloids-9.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-100.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-101.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-102.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-103.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-104.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-105.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-106.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthom

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.layers import Dense,Flatten,Input,LSTM,Convolution2D,MaxPooling2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import numpy as np

In [8]:
train_gen = ImageDataGenerator(rescale=(1./255),horizontal_flip=True,shear_range=0.2)
test_gen = ImageDataGenerator(rescale=(1./255))

In [9]:
train = train_gen.flow_from_directory('/content/test', target_size=(224,224), class_mode='categorical', batch_size=32,shuffle=True)
test = test_gen.flow_from_directory('/content/test', target_size=(224,224), class_mode='categorical', batch_size=32,shuffle=True)

Found 4002 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


In [10]:
train.class_indices

{'Acne and Rosacea Photos': 0,
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions': 1,
 'Atopic Dermatitis Photos': 2,
 'Bullous Disease Photos': 3,
 'Cellulitis Impetigo and other Bacterial Infections': 4,
 'Eczema Photos': 5,
 'Exanthems and Drug Eruptions': 6,
 'Hair Loss Photos Alopecia and other Hair Diseases': 7,
 'Herpes HPV and other STDs Photos': 8,
 'Light Diseases and Disorders of Pigmentation': 9,
 'Lupus and other Connective Tissue diseases': 10,
 'Melanoma Skin Cancer Nevi and Moles': 11,
 'Nail Fungus and other Nail Disease': 12,
 'Poison Ivy Photos and other Contact Dermatitis': 13,
 'Psoriasis pictures Lichen Planus and related diseases': 14,
 'Scabies Lyme Disease and other Infestations and Bites': 15,
 'Seborrheic Keratoses and other Benign Tumors': 16,
 'Systemic Disease': 17,
 'Tinea Ringworm Candidiasis and other Fungal Infections': 18,
 'Urticaria Hives': 19,
 'Vascular Tumors': 20,
 'Vasculitis Photos': 21,
 'Warts Molluscum and other Viral Inf

In [11]:
#Input
vgg=VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))
for layer in vgg.layers:
  layer.trainable=False
x=Flatten()(vgg.output)
output=Dense(23,activation='softmax')(x)

58889256/58889256 [==============================] - 2s 0us/step


In [12]:
model = Model(inputs=vgg.input,outputs=output)

In [13]:
early_stop = EarlyStopping(monitor='accuracy', patience=8)

In [14]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
model.fit(train,validation_data=test,epochs=30,batch_size=20,callbacks=early_stop)

Epoch 1/30
126/126 [==============================] - 134s 958ms/step - loss: 3.2161 - accuracy: 0.1804 - val_loss: 2.4968 - val_accuracy: 0.3296
Epoch 2/30
126/126 [==============================] - 108s 862ms/step - loss: 2.4170 - accuracy: 0.3311 - val_loss: 1.7743 - val_accuracy: 0.5100
Epoch 3/30
126/126 [==============================] - 115s 915ms/step - loss: 1.9768 - accuracy: 0.4340 - val_loss: 1.6229 - val_accuracy: 0.5370
Epoch 4/30
126/126 [==============================] - 108s 860ms/step - loss: 1.7148 - accuracy: 0.5065 - val_loss: 1.3544 - val_accuracy: 0.6022
Epoch 5/30
126/126 [==============================] - 114s 906ms/step - loss: 1.5059 - accuracy: 0.5595 - val_loss: 1.3850 - val_accuracy: 0.5792
Epoch 6/30
126/126 [==============================] - 109s 867ms/step - loss: 1.4113 - accuracy: 0.5985 - val_loss: 1.1606 - val_accuracy: 0.6664
Epoch 7/30
126/126 [==============================] - 109s 867ms/step - loss: 1.1867 - accuracy: 0.6579 - val_loss: 1.0429 -

In [ ]:
model.save('skindiseases.h5')

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
img1 = image.load_img('/content/ecz.jpeg',target_size=(224,224))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred1 = np.argmax(model.predict(img1))
output = ['Acne and Rosacea Photos','Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions','Atopic Dermatitis Photos',
          'Bullous Disease Photos','Cellulitis Impetigo and other Bacterial Infections','Eczema Photos','Exanthems and Drug Eruptions',
          'Hair Loss Photos Alopecia and other Hair Diseases','Herpes HPV and other STDs Photos','Light Diseases and Disorders of Pigmentation',
          'Lupus and other Connective Tissue diseases','Melanoma Skin Cancer Nevi and Moles','Nail Fungus and other Nail Disease',
          'Poison Ivy Photos and other Contact Dermatitis','Psoriasis pictures Lichen Planus and related diseases',
          'Scabies Lyme Disease and other Infestations and Bites','Seborrheic Keratoses and other Benign Tumors','Systemic Disease',
          'Tinea Ringworm Candidiasis and other Fungal Infections','Urticaria Hives','Vascular Tumors','Vasculitis Photos','Warts Molluscum and other Viral Infections']
print(output[pred1])

1/1 [==============================] - 1s 681ms/step
Eczema Photos


In [ ]:
img2 = image.load_img('/content/ak.jpeg',target_size=(224,224))
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2,axis=0)
pred2 = np.argmax(model.predict(img2))
output = ['Acne and Rosacea Photos','Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions','Atopic Dermatitis Photos',
          'Bullous Disease Photos','Cellulitis Impetigo and other Bacterial Infections','Eczema Photos','Exanthems and Drug Eruptions',
          'Hair Loss Photos Alopecia and other Hair Diseases','Herpes HPV and other STDs Photos','Light Diseases and Disorders of Pigmentation',
          'Lupus and other Connective Tissue diseases','Melanoma Skin Cancer Nevi and Moles','Nail Fungus and other Nail Disease',
          'Poison Ivy Photos and other Contact Dermatitis','Psoriasis pictures Lichen Planus and related diseases',
          'Scabies Lyme Disease and other Infestations and Bites','Seborrheic Keratoses and other Benign Tumors','Systemic Disease',
          'Tinea Ringworm Candidiasis and other Fungal Infections','Urticaria Hives','Vascular Tumors','Vasculitis Photos','Warts Molluscum and other Viral Infections']
print(output[pred2])

1/1 [==============================] - 1s 517ms/step
Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions
